In [23]:
#Si el script tira el siguiente error: 
#AttributeError: module 'win32com.gen_py.00020813-0000-0000-C000-000000000046x0x1x6' has no attribute 'CLSIDToPackageMap'
#Borrar archivos temporales en:
#import win32com
#print(win32com.__gen_path__)
import odbc
import win32com.client
import pandas as pd
import pyodbc
import os

from win32com.client.gencache import EnsureDispatch

def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [24]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel where end_processing_date is null'

df = pd.read_sql(query,cnxn)

In [25]:
df

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [26]:
def check_excel(file):
    
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'

    try:
        xl = win32com.client.Dispatch('Excel.Application')
    except AttributeError as ae:
        print(ae)
        print("BORRAR TEMPORALES")
    try:
        wb = xl.Workbooks.Open(file, None,True)
    except Exception as e:
        #print(e)
        #xl.Application.Quit()
        return e , []
        
    datos = []
    try:
        for x in wb.Connections:
            for i in x.Ranges:
                try:
                    for p in i.ListObject.QueryTable.Parameters:
                        datos.append([x,
                                      x.ODBCConnection.CommandText,
                                      x.ODBCConnection.Connection,
                                      i.Parent.Name,
                                      i.Cells(1,1)[2],
                                      p.SourceRange.Address,
                                      p.SourceRange.Parent.Name,
                                      p.SourceRange.Value])

                except Exception as e:
                    try:
                        datos.append([x,
                                      x.ODBCConnection.CommandText,
                                      x.ODBCConnection.Connection,
                                      i.Parent.Name,
                                      i.Cells(1,1)[2],
                                      None,
                                      None,
                                      None])
                    except:
                        try:
                            for p in i.ListObject.QueryTable.Parameters:
                                datos.append([x,
                                              x.OLEDBConnection.CommandText,
                                              x.OLEDBConnection.Connection,
                                              i.Parent.Name,
                                              i.Cells(1,1)[2],
                                              p.SourceRange.Address,
                                              p.SourceRange.Parent.Name,
                                              p.SourceRange.Value])
                        except:
                            datos.append([x,
                                          x.OLEDBConnection.CommandText,
                                          x.OLEDBConnection.Connection,
                                          i.Parent.Name,
                                          i.Cells(1,1)[2],
                                          None,
                                          None,
                                          None])

    except Exception as c:
        return c , []
    

    #df = pd.DataFrame(datos)
    #df = df.applymap(str)
    #datos = map(str,datos)
    if datos == []:
        datos.append(['None',
                      'None',
                      'None',
                      'None',
                      'None',
                      'None',
                      'None',
                      'None'])
    elif type(datos[0]) == list:
        datos = [list(map(str,i))for i in datos]
    
    else:
        datos = list(map(str,datos))
    xl.DisplayAlerts = False
    wb.Close(0)
    xl.Application.Quit()
    
    #return df
    return 0, datos

In [5]:
def insert_file_connection(EXCEL_ID,NAME,STR_CONN,SQL):
    
    aux_cnn = connection()
    
    cursor = aux_cnn.cursor()
    
    params = EXCEL_ID,NAME,STR_CONN,SQL
    
    #query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%params
    query = """EXEC planeamiento..ctrl_excel_InsertFileConnection  %s, '%s', '%s', '%s'"""%params
    #sql = "{CALL planeamiento..ctrl_excel_InsertFileConnection (?,?,?,?)}"
    
    try:
        #cursor.execute(sql, params)
        cursor.execute(query)
        val = cursor.fetchval()
        cursor.commit()
        
        
        
    except Exception as e:
        print('*'*100)
        print(e)
        
    
    aux_cnn.close()
    return int(val)
    
    
    

In [27]:
#x = insert_file_connection(5,'PROBANDO NAME','CONEXION PROBANDO','SELECT * FROM PRUEBA')

In [ ]:
x

In [ ]:
cursor = cnxn.cursor()
for index,row in df.iterrows():                                                          
    cursor.execute("INSERT INTO tabla(,,,]) values (?,?,?)",
                   row['col'])
    cnxn.commit()
cursor.close()
cnxn.close()


datos = []



In [1]:
#PROBANDO CON SQLALCHEMY
from sqlalchemy import create_engine
import urllib
server = 'tcp:192.168.1.6'
database = 'planeamiento'
username = 'sa'
password = 'sa'

params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER='+
                                 server+';DATABASE='+database+';UID='+username+';PWD='+ password)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [2]:
EXCEL_ID = 5
NAME ='PROBANDO NAME'
STR_CONN = 'CONEXION PROBANDO'
SQL = 'SELECT * FROM PRUEBA'

In [3]:
parametros = (EXCEL_ID,NAME,STR_CONN,SQL)
query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%parametros

In [16]:
import pandas as pd

In [6]:
#parametros = (EXCEL_ID,NAME,STR_CONN,SQL)
    
#query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = {0}, @name = {1}, @str_conn = {2}, @sql = {3}".format(params)
    
#query = 'EXEC my_procedure @Param = {0}'.format(my_param)

In [7]:
df = pd.read_sql_query(query, engine)
#*** HASTA SQLALCHEMY

In [8]:
df

,ctrl_excel_connection_id
0,111.0


In [15]:
PPP= df.loc[2]['path']+df.loc[2]['filename']

In [75]:
# 1 - creo un df con la query 'select * from planeamiento..ctrl_excel where end_processing_date is null'
# 2 - recorro el df leyendo cada excel en cada vuelta del loop llama a una función que devuelve todos los parametros (cadena de conexion,query, nombre de la query, parametros etc.)
# 3 - cada lista que devuelve la función la inserto en la base de datos. ( llamada a procedures)

# Estos son lo procedures:
#     planeamiento..ctrl_excel_InsertFileConnection (Devuelve un excel_connection_id para insertar en la proxima tabla)
#     planeamiento..ctrl_excel_InsertFileConnectionLocation (Devuelve un excel_connection_location_id para insertar en la prox tabla)
#     planeamiento..ctrl_excel_InsertFileConnectionParameter (Devuelve excel_connection_location_parameter_id)
    
    


In [53]:
import os
from glob import glob
archivos = []
PATH = "C:\\Users\\jhernandez\\Desktop"
EXT = ["*.xls","*.xlsx","*.xlsm"]
files = []
for x in EXT:
    a = [file
                     for path, subdir, files in os.walk(PATH)
                     for file in glob(os.path.join(path, x))]
    files.append(a)

files

[['C:\\Users\\jhernandez\\Desktop\\Calidad IMPRESION ROTULOS.xls',
  'C:\\Users\\jhernandez\\Desktop\\excels\\Calidad IMPRESION ROTULOS.xls'],
 ['C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
  'C:\\

In [46]:
archivos

['C:\\Users\\jhernandez\\Desktop\\Canjes electros.xlsm',
 'C:\\Users\\jhernandez\\Desktop\\excels\\macro 2.xlsm',
 'C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\excels\\130120

In [47]:
import os
from glob import glob
archivos = []
PATH = "C:\\Users\\jhernandez\\Desktop"
EXT = "*.xls"

xls = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
EXT = "*.xlsx"
xlsx = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]

EXT = "*.xlsm"
xlsm = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]

archivos = xls + xlsx +xlsm

In [48]:
list(archivos)

['C:\\Users\\jhernandez\\Desktop\\Calidad IMPRESION ROTULOS.xls',
 'C:\\Users\\jhernandez\\Desktop\\excels\\Calidad IMPRESION ROTULOS.xls',
 'C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernande

In [ ]:
for x in archivos:
    cont = 0
    #iteracion por cada x con check_excel
    # if check_excel(x)["parametros"] != null:
        #hacer un insert en: select * from planeamiento..ctrl_excel_connection_parameter 
            #ELSE: cont +1
    # if check_excel(x)["string_connection"]!= null:
        #hacer un insert en:select * from planeamiento..ctrl_excel_connection 
            #ELSE: cont +1
    #if check_excel(x)["text_command"]!= null:
        #hacer un insert en:select * from planeamiento..ctrl_excel_connection_location 
            #ELSE: cont +1
    #if cont > 0:
    #  insertar ese excel en una nueva tabla(excels candidatos a borrar) y describir que parametros le falta F:
    print(x)

In [27]:
import datetime
#datetime.datetime.today()

In [29]:
datos = []
errores = []
start = datetime.datetime.today()
for index,row in df.iterrows():
    
    archivo = row['path']+row['filename']
    result = check_excel(archivo)
    if result[0] != 0:
        
        errores.append([row['excel_id'],archivo,result[0]])
        
            
    else:
        respuesta = result[0]
        if result[1] != []:
            
            if type(result[1]) == list:
                for res in result[1]:
                    res.append(row['excel_id'])
                    res.append(index)
                
            else:
                result[1].append(row['excel_id'])
                result[1].append(index)
            datos += result[1]
            #raise
    if index == 100:
        
        end = datetime.datetime.today()
        print(end-start)
        import os
        os.system("taskkill /EXCEL.EXE")
        raise
#     respuesta = result[0]
#     result[1]

0:02:54.077813


RuntimeError: No active exception to reraise

In [31]:
os.system("taskkill /f /EXCEL.EXE")

1

In [191]:
end = datetime.datetime.today() #0:08:50.505707
print(end-start)

0:08:50.505707


In [16]:
errores[0]

[42,
 'Z:\\....Candidatos a borrar\\Archivos Viejos\\TODO\\IMPRESION ROTULOS DEPOSITO.xls',
 pywintypes.com_error(-2147352567,
                      'Ocurrió una excepción.',
                      (0, None, None, None, 0, -2147467263),
                      None)]

In [17]:
# 42
errores[-1]

[140,
 'Z:\\Consultas\\Catálogos\\03- 2014\\1403-1404\\Listado 1403-1404 versión 2 - con bombachas.xlsx',
 pywintypes.com_error(-2147352567,
                      'Ocurrió una excepción.',
                      (0, None, None, None, 0, -2147467263),
                      None)]

In [22]:
datos

[['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 40, 0],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 41, 1],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 43, 3],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 46, 6],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 47, 7],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 48, 8],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 49, 9],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 50, 10],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 51, 11],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 52, 12],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 54, 14],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 56, 16],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 57, 17],
 ['None', 'None', 'None', 'None'

In [21]:
df['excel_id'].unique().tolist()

[40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216

In [171]:
len(errores) #23

In [86]:
index

100

In [28]:
row['path']+row['filename']

'Z:\\Consultas\\Calendarios\\en desuso\\Circuitos Zonas.xlsx'

In [18]:
datos = []
errores = []
start = datetime.datetime.today()

    
archivo = row['path']+row['filename']
result = check_excel(archivo)
if result[0] != 0:

    errores.append([row['excel_id'],archivo,result[0]])


else:
    respuesta = result[0]
    if result[1] != []:

        if type(result[1]) == list:
            for res in result[1]:
                res.append(row['excel_id'])
                res.append(index)

        else:
            result[1].append(row['excel_id'])
            result[1].append(index)
        datos += result[1]
        #raise
if index == 100:

    end = datetime.datetime.today()
    print(end-start)
    raise
#     respuesta = result[0]
#     result[1]

AttributeError: module 'win32com.gen_py.00020813-0000-0000-C000-000000000046x0x1x6' has no attribute 'CLSIDToPackageMap'

In [179]:
#**DEBUG
#xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
#file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
xl = win32com.client.Dispatch('Excel.Application')
file= row['path']+row['filename']
try:
    wb = xl.Workbooks.Open(file, None,True)
except Exception as e:
    print(e)
    

datos = []

In [183]:
print(row['path']+row['filename'])

Z:\....Candidatos a borrar\Archivos Viejos\Observaciones.xlsm


In [180]:
Q=[]
for x in wb.Connections:
    for i in x.Ranges:
        try:
            for p in i.ListObject.QueryTable.Parameters:
                Q.append([x,
                              x.ODBCConnection.CommandText,
                              x.ODBCConnection.Connection,
                              i.Parent.Name,
                              i.Cells(1,1)[2],
                              p.SourceRange.Address,
                              p.SourceRange.Parent.Name,
                              p.SourceRange.Value])

        except Exception as e:
            try:
                Q.append([x,
                              x.ODBCConnection.CommandText,
                              x.ODBCConnection.Connection,
                              i.Parent.Name,
                              i.Cells(1,1)[2],
                              None,
                              None,
                              None])
            except:
                try:
                    for p in i.ListObject.QueryTable.Parameters:
                        Q.append([x,
                                      x.OLEDBConnection.CommandText,
                                      x.OLEDBConnection.Connection,
                                      i.Parent.Name,
                                      i.Cells(1,1)[2],
                                      p.SourceRange.Address,
                                      p.SourceRange.Parent.Name,
                                      p.SourceRange.Value])
                except:
                    Q.append([x,
                                  x.OLEDBConnection.CommandText,
                                  x.OLEDBConnection.Connection,
                                  i.Parent.Name,
                                  i.Cells(1,1)[2],
                                  None,
                                  None,
                                  None])

In [181]:
Q

[]

In [34]:
result

(0,
 [['Consulta desde ESTIMACIONES',
   'SELECT vFechasProcesos2.g, vFechasProcesos2.cp, vFechasProcesos2.zona, vFechasProcesos2.segmento, vFechasProcesos2.categoria, vFechasProcesos2.estadoSIS, vFechasProcesos2.sistemaSIS, vFechasProcesos2.fechaCalendario, vFechasProcesos2.fechaReal, vFechasProcesos2.UnidadesIngresadas, vFechasProcesos2.PedidosIngresados, vFechasProcesos2.fechaLiberada, vFechasProcesos2.fechaRetenida, vFechasProcesos2.fechaDesarme, vFechasProcesos2.estadoActual, vFechasProcesos2.despacho72Hs, vFechasProcesos2.despachadoReal, vFechasProcesos2.Retraso, vFechasProcesos2.tipoEnvio, vFechasProcesos2.fechaProceso, vFechasProcesos2.areaResponsable, vFechasProcesos2.estadoProceso, vFechasProcesos2.fechaUltimoIngreso, vFechasProcesos2.fechaBajada, vFechasProcesos2.UltimaActualizacion, vFechasProcesos2.ubicacion, vFechasProcesos2.prioridadfecha, vFechasProcesos2.prioridadtipo, vFechasProcesos2.ComentarioRegional, vFechasProcesos2.ComentarioJB,vFechasProcesos2.catalogos\r\nFROM

In [157]:
index

In [8]:
len(datos)

61

In [9]:
len(errores)

40

In [59]:
row['path']+row['filename']

'Z:\\Consultas\\Catálogos\\03- 2014\\1403-1404\\Listado 1403-1404 versión 2 - con bombachas.xlsx'

In [62]:
df[df['excel_id']]

['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 107, 67]

In [63]:
100-65

35

In [64]:
df.loc[65]

excel_id                                                 105
path                     Z:\Consultas\Calendarios\en desuso\
filename                           Calendario  2019 ult.xlsx
date_of_insertion                 2020-01-23 12:10:15.320000
start_processing_date                                    NaT
end_processing_date                                     None
process_id                                              None
error_msg                                               None
Name: 65, dtype: object

In [17]:
errores

[]

In [89]:
df[df['excel_id']==140]

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
100,140,Z:\Consultas\Catálogos\03- 2014\1403-1404\,Listado 1403-1404 versión 2 - con bombachas.xlsx,2020-01-23 12:10:16.743,NaT,None,None,None


In [68]:
df.loc[64:]

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
64,104,Z:\Consultas\Calendarios\en desuso\,Calendario 2019 DIEGO.xlsx,2020-01-23 12:10:15.310,NaT,None,None,None
65,105,Z:\Consultas\Calendarios\en desuso\,Calendario 2019 ult.xlsx,2020-01-23 12:10:15.320,NaT,None,None,None
66,106,Z:\Consultas\Calendarios\en desuso\,Circuitos Zonas.xlsx,2020-01-23 12:10:15.327,NaT,None,None,None
67,107,Z:\Consultas\Calendarios\en desuso\,Copia de seguridad de Calendario 2014 Delis.xlsx,2020-01-23 12:10:15.337,NaT,None,None,None
68,108,Z:\Consultas\Calendarios\en desuso\,despacho cat electro.xlsx,2020-01-23 12:10:15.343,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [16]:
datos[-1]

IndexError: list index out of range

In [8]:
import win32com
print(win32com.__gen_path__)

C:\Users\JHERNA~1\AppData\Local\Temp\gen_py\3.7
